# Preprocessing

In [ ]:
# from google.colab import files

In [ ]:
# uploaded = files.upload()

In [ ]:
import numpy as np
import pandas as pd
import json
import torch
from torch.utils.data import Dataset, DataLoader
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import brown
nltk.download('punkt')
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.models.word2vec import Word2Vec
from sklearn.model_selection import train_test_split
import gensim
import logging
from tqdm import tqdm

np.random.seed(0)
torch.manual_seed(0)

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/bharatsuri/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

model = gensim.models.keyedvectors.Word2VecKeyedVectors.load_word2vec_format('glove.42B.300d.w2vformat.txt')

2021-03-29 00:25:41,274 : INFO : loading projection weights from glove.42B.300d.w2vformat.txt
2021-03-29 00:29:14,440 : INFO : loaded (1917494, 300) matrix from glove.42B.300d.w2vformat.txt


In [ ]:
def getSentenceVector(sentence):
    words = word_tokenize(sentence)
    count_present = 0
    vec = np.zeros(300)
    for word in words:
        if word in model.wv:
            vec = np.add(vec, np.array(model.wv[word]))
            count_present += 1
    if count_present > 0:
        vec = vec / count_present
    return vec

In [ ]:
class YelpDataset(Dataset):
    """Yelp dataset."""
    
    def __init__(self, file_name, append_zeros = True):
        """
        Args:
            file_name: The json file to make the dataset from
        """
        self.df = pd.read_json(file_name, lines=True)

        tensors = []
        binary_cat = []
        zero_sentence = np.zeros(300)

        #Create target class and document vector for each review
        for index, row in self.df.iterrows():
            single_class = np.zeros(2)
            if row['category'] == 1:
              single_class[1] = 1
            else:
              single_class[0] = 1
            binary_cat.append(torch.tensor(single_class))

            sentences = sent_tokenize(row['text'])
            sent_vecs = []
            for i in range(20):
              if i < len(sentences):
                sent_vecs.append(getSentenceVector(sentences[i]))
              elif append_zeros:
                sent_vecs.append(zero_sentence.copy())
            tensors.append(torch.FloatTensor(sent_vecs))

        self.df['category'] = binary_cat
        self.df['vector'] = tensors
        del self.df['text']

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        category = self.df.iloc[idx, 0]
        vector = self.df.iloc[idx, 1]
        sample = {'vector': vector, 'category': category}

        return sample

In [ ]:
### DO NOT APPEND ZEROS ###
#append_zeros = False
dataset_train = YelpDataset('dataset/dataset_train.json', False)
dataset_dev = YelpDataset('dataset/dataset_dev.json', False)
dataset_test = YelpDataset('dataset/dataset_test.json', False)

<ipython-input-5-9dc5e6bd6841>:6: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if word in model.wv:
<ipython-input-5-9dc5e6bd6841>:7: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  vec = np.add(vec, np.array(model.wv[word]))
<ipython-input-5-9dc5e6bd6841>:6: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if word in model.wv:
<ipython-input-5-9dc5e6bd6841>:7: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  vec = np.add(vec, np.array(model.wv[word]))
<ipython-input-5-9dc5e6bd6841>:6: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if word in model.wv:
<ipython-input-5-9dc5e6bd6841>:7: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  vec = np.add(vec, np.array(model.

In [ ]:
dataloader_train = DataLoader(dataset_train, batch_size=1, shuffle=True, 
                              num_workers=0)
dataloader_dev = DataLoader(dataset_dev, batch_size=1, shuffle=True, 
                              num_workers=0)
dataloader_test = DataLoader(dataset_test, batch_size=1, shuffle=True, 
                              num_workers=0)

In [ ]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
class GRUAttention(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(GRUAttention, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.gru = nn.GRU(input_size, hidden_size, batch_first=True, bidirectional=True)
        
        self.fcn = nn.Sequential(
            nn.Linear(2*hidden_size, 64),
            nn.Tanh(),
            nn.Dropout(0.8),
            nn.Linear(64, 2),
            nn.Tanh()
        )
    
    def attention(self, outputs, hidden):
        h = hidden.squeeze(0)
        attn_weights = torch.bmm(outputs, h.unsqueeze(2)).squeeze(2)
        attn_scores = F.softmax(attn_weights, 1)
        s = torch.bmm(outputs.transpose(1, 2), attn_scores.unsqueeze(2)).squeeze(2)
        return s, attn_scores

    def forward(self, inputs):
        output, hidden = self.gru(inputs)
        hidden = torch.cat((hidden[0], hidden[1]), 1).unsqueeze(0)
        out, scores = self.attention(output, hidden)
        logits = self.fcn(out)
        return logits, scores

In [ ]:
from tqdm import tqdm
encoder = GRUAttention(300, 64)
criterion = nn.CrossEntropyLoss()

encoder_optimizer = optim.Adam(encoder.parameters(), lr=0.0001)

epochs = 25
total = 0
for n in range(epochs):
    epoch_loss = 0
    for sample_batched in tqdm(dataloader_train):
        encoder.train()
        encoder.zero_grad()
        loss = 0
        seq = sample_batched['vector']
        output, scores = encoder(seq)
        output = output.view(1, -1)
        arg_max = torch.argmax(sample_batched['category'][0])
        target = torch.tensor([arg_max])
        loss += criterion(output, target)
        epoch_loss+=loss.detach().item()
        loss.backward()

        encoder_optimizer.step()

    print("Average loss at epoch {}: {}".format(n, epoch_loss/len(dataloader_train)))
    validate()



  2%|▏         | 227/10000 [00:00<00:08, 1129.56it/s]

Average loss at epoch 0: 0.38486465228706596


  0%|          | 17/50000 [00:00<05:02, 164.97it/s]

Accuracy: 0.8746


  2%|▏         | 242/10000 [00:00<00:08, 1212.04it/s]

...

Accuracy: 0.8969


  2%|▏         | 248/10000 [00:00<00:07, 1242.01it/s]

Average loss at epoch 24: 0.2160248425680399


100%|██████████| 10000/10000 [00:08<00:00, 1219.72it/s]

Accuracy: 0.8956


In [ ]:
encoder.eval()
with torch.no_grad():
    total_correct = 0
    total = 0
    for sample_batched in tqdm(dataloader_train):

        loss = 0
        output, scores = encoder(sample_batched['vector'])
        classification = torch.argmax(output)
        arg_max = torch.argmax(sample_batched['category'][0])
        if classification == arg_max:
            total_correct+=1
    print("Accuracy: {}".format(total_correct/len(dataloader_train)))

In [ ]:
def validate():
    encoder.eval()
    total_correct = 0
    total = 0
    with torch.no_grad():
        for sample_batched in tqdm(dataloader_dev):

            loss = 0
            output, scores = encoder(sample_batched['vector'])
            classification = torch.argmax(output)
            arg_max = torch.argmax(sample_batched['category'][0])
            if classification == arg_max:
                total_correct+=1
    print("Accuracy: {}".format(total_correct/len(dataloader_dev)))